# Linear interpolation of magnetic anomaly data

I was going to use [Verde spline interpolator](https://www.fatiando.org/verde/latest/gallery/spline.html?highlight=regular), but it gave me a MemoryError. The same happened with most of scipy interpolators, like [interp2d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp2d.html) and [griddata](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html).

Trying to use scipy [generic_filter](https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.ndimage.filters.generic_filter.html) to interpolate in moving windows with these interpolators was too slow to yield any results.

I tested several (maybe all) algorithms and functions available on scipy and other python API. Unfortunately, none of them were able to take advantage of a regular grid with missing values for interpolation.

Nevertheless, [LinearNDInterpolator](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.LinearNDInterpolator.html) from scipy did the job with a good performance.

In [ ]:
import matplotlib.pyplot as plt
import verde as vd
%matplotlib inline
import seaborn as sns
sns.set_style('ticks')

import numpy as np

In [ ]:
# Load the data as read only
data = np.load('../data/interim/MAGIGRF_DECIMATED_150x150m.npy', mmap_mode='r')

This shall be the final interpolated grid size

In [ ]:
x = np.unique(data['x']) # Grid x coordinates
y = np.unique(data['y']) # Grid y coordinates

print('Grid size: {0} x {1}'.format(x.shape[0], y.shape[0]))

In [ ]:
# Start with a sample blank image to see where the holes are
shape = (x.shape[0], y.shape[0])
img = np.full(shape, np.nan)

In [ ]:
# Fill the bins as needed
img[np.digitize(data['x'], x) - 1, np.digitize(data['y'], y) - 1] = data['data']

In [ ]:
vmin, vmax = np.percentile(data['data'], [2,98])

plt.figure(figsize=(10,7))
ax = plt.subplot(111)
extent=[x.min(),x.max(), y.min(), y.max()]
ax.set_title('Non interpolated magnetic field')
im = ax.imshow(img.T, interpolation='none', aspect='auto',
           vmin=vmin, vmax=vmax, cmap='viridis',
           extent=extent)
plt.colorbar(im, shrink=0.75, label='magnetic anomaly (nT)')
plt.ylim(plt.ylim()[::-1])
ax.set_aspect('equal', 'datalim')

plt.xlabel('easting (m)')
plt.xlabel('northing (m)')
plt.tight_layout();

Note: Some pixels appear missing due to the image size on the plot

## Linear 2D interpolation

In [ ]:
from scipy.interpolate import LinearNDInterpolator

In [ ]:
%%time

# I'll use a linear interpolator with Delaunay triangulation
linear_interpolator = LinearNDInterpolator((data['x'], data['y']), data['data'], fill_value=np.nan)

In [ ]:
%%time
# Coordinates for interpolation
X, Y = np.meshgrid(x,y)

interpolated = linear_interpolator(X,Y)

In [ ]:
def create_mask(row):
    retval = np.full(row.shape, True)
    
    i =  np.where(~np.isnan(row))[0]
    n = len(i)
    
    if n == 0:
        return retval
    
    elif n == 1:
        retval[i[0]] = False
        return retval
    
    elif n >= 2:
        retval[i[0]:i[-1]] = False
        return retval
    
mask = np.array([create_mask(col) for col in img.T])
mask = np.logical_or(mask, np.isnan(interpolated))

plt.figure(figsize=(10,7))
ax = plt.subplot(111)
extent=[x.min(),x.max(), y.min(), y.max()]
ax.set_title('Final interpolation mask')
im = ax.imshow(mask.astype(np.int), interpolation='none', aspect='auto',
               extent=extent)
plt.colorbar(im, shrink=0.75)
plt.ylim(plt.ylim()[::-1])
ax.set_aspect('equal', 'datalim')
plt.xlabel('easting (m)')
plt.xlabel('northing (m)')
plt.tight_layout();

In [ ]:
# Masked final linear interpolated magnetic field
final = np.ma.array(interpolated, mask=mask)

In [ ]:
from matplotlib.colors import LightSource

In [ ]:
# Cell size in meters
dx, dy = np.mean(np.diff(x)), np.mean(np.diff(y))
dx, dy

In [ ]:
def plot_hillshaded_image(image, vert_exag=20, blend_mode='soft',
                          cmap=plt.cm.viridis, dx=dx, dy=dy,
                          vmin=None, vmax=None, title=None, extent=extent,
                          save_path=None):
    
    # Shade from the northwest, with the sun 45 degrees from horizontal
    ls = LightSource(azdeg=30, altdeg=45)
    cmap = cmap

    fig = plt.figure(figsize=(10,7))
    ax = plt.subplot(111)
    
    if title is not None:
        ax.set_title(title)

    rgb = ls.shade(final, cmap=cmap, blend_mode='soft', vert_exag=vert_exag,
                   dx=dx, dy=dy, vmin=vmin, vmax=vmax)

    # Use a proxy artist for the colorbar
    im = ax.imshow(final, cmap=cmap, vmin=vmin, vmax=vmax)
    im.remove()
    fig.colorbar(im, shrink=0.75, label='magnetic anomaly (nT)')
    ax.imshow(rgb, interpolation='none', aspect='auto',extent=extent)
    
    plt.ylim(plt.ylim()[::-1])
    ax.set_aspect('equal', 'datalim')
    plt.xlabel('easting (m)')
    plt.ylabel('northing (m)')
    plt.tight_layout()
    
    if save_path is not None:
        plt.savefig(save_path, dpi=300)
    plt.show()
    plt.close();
    
plot_hillshaded_image(final, vmin=vmin, vmax=vmax, extent=extent, title='Linear interpolated magnetic anomaly',
                     save_path='../reports/figures/1113_MAGIGRF.png')

## Writing the final values to a GeoTiff

In [ ]:
import rasterio
from rasterio.transform import from_origin

In [ ]:
transform = from_origin(x.min(), y.min(), dx, -dy)
crs = rasterio.crs.CRS.from_epsg(32722)

nodata = -99999 # No data value

new_dataset = rasterio.open('../data/processed/1113_MAGIGRF.tif', 'w', driver='GTiff',
                            height = final.shape[0], width = final.shape[1],
                            count=1, dtype=str(final.dtype),
                            crs=crs,
                            transform=transform, nodata=nodata)

In [ ]:
new_dataset.write(final.filled(nodata), 1)
new_dataset.close()